In [1]:
# yield from可用于简化for循环中的yield表达式。

def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

list(gen())

['A', 'B', 1, 2]

In [2]:
def gen():
    yield from 'AB'
    yield from range(1, 3)

list(gen())

['A', 'B', 1, 2]

In [3]:
# 使用yield from链接可迭代的对象

def chain(*iterables):
    for it in iterables:
        yield from it

s = 'ABC'
t = tuple(range(3))

list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [7]:
#  coroaverager3.py：使用yield from计算平均值并输出统计报告

from collections import namedtuple
Result = namedtuple('Result', 'count average')

# 子生成器
# 这里作为子生成器使用。
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        # main函数中的客户代码发送的各个值绑定到这里的term变量上。
        term = yield
        #  至关重要的终止条件。如果不这么做，使用yield from调用这个协程的生成器会永远阻塞。
        if term is None:
            break
        total+= term
        count+= 1
        average = total/count
        # 返回的Result会成为grouper函数中yield from表达式的值。
    return Result(count, average)

# 委派生成器
# grouper是委派生成器
def grouper(results, key):
    # 这个循环每次迭代时会新建一个averager实例；每个实例都是作为协程使用的生成器对象。
    while True:
        # grouper发送的每个值都会经由yield from处理，通过管道传给averager实例。
        results[key] = yield from averager()

#客户端代码，即调用方
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)  #重要
    # print(results)  #如果要调试，去掉注释
    report(results)

#输出报告
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
            result.count, group, result.average, unit))

data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}
main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m
